# Animating End-to-End Network Actication Maps

Authors: Emily Kaczmarek, Olivier Miguel, Kevin Dick

---

#### Resources:
* GitHub repo to push our changes to: https://github.com/jacobgil/pytorch-grad-cam
* Command for ffmpeg: `ffmpeg -framerate 5 -pattern_type glob -i "*.png" -c:v libx264 -r 30 -vf "scale=1200:-2,format=yuv420p" -movflags +faststart output.mp4`

TODO: write fancy things here

#### Hackathon Day 1:
* Determine a few example benchmark image datasets that we demo
  * MNIST
  * whatever GradCAM paper used
  * Anotherrrrr
* Determine what pre-trained model architecture we use:
 * DenseNet (no isses in layer name



# Enironment setup

## Colab

For colab we need to:
1. clone the repo
2. Define google drive location

We connect google drive to save the generated CAManim picutres and videos. The root of the save directory is defined by the variable `save_root`

In [ ]:
!git clone https://github.com/OMNI-ML/pytorch-grad-cam-anim.git
!ls

In [ ]:
# the % command makes the cd last beyond this cell / line
%cd pytorch-grad-cam-anim
!ls
!git checkout adapt-basecam-to-support-cam_anim

In [ ]:
# useful to check the last commits and ensure recent changes are there
!git log --oneline --max-count=10 # --reverse

In [ ]:
%load_ext memory_profiler

## Output Directory

In [ ]:
from pathlib import Path

### Colab + GDrive

In [ ]:
from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
drive.mount("/content/gdrive")
drive_path = '/content/gdrive/MyDrive'

### local

In [ ]:
drive_path = '.'
save_root = Path(drive_path) / "starCAManim"
save_root.mkdir(parents=True, exist_ok=True)
print(f"save_root: {save_root.resolve()}")

## Base

Base packages / requirements needed to run notebook

In [ ]:
# conda create --name myenv python=3

In [ ]:
# when you run this notebook locally, the current directory is the one where the notebook is.
# this command changes the current directory to the repository root
%cd ../
!dir

In [ ]:
# !python setup.py install - avoid this
!pip install . #-e ??
!pip install -r requirements_CAManim.txt
# !pip install ffmpeg-python

In [ ]:
bundles_dir = Path("./bundles")

In [ ]:
bundles_dir = Path("./bundles")
!python -m monai.bundle download "breast_density_classification" --bundle_dir "./bundles/"

In [ ]:
import json
import numpy as np
import cv2
from PIL import Image
import requests
from pytorch_grad_cam.utils.image import preprocess_image
from tqdm import tqdm
import torch
import pandas as pd

from monai.networks.nets import TorchVisionFCModel

# Load image/data

bear image

In [ ]:
image_url = "https://th.bing.com/th/id/R.94b33a074b9ceeb27b1c7fba0f66db74?rik=wN27mvigyFlXGg&riu=http%3a%2f%2fimages5.fanpop.com%2fimage%2fphotos%2f31400000%2fBear-Wallpaper-bears-31446777-1600-1200.jpg&ehk=oD0JPpRVTZZ6yizZtGQtnsBGK2pAap2xv3sU3A4bIMc%3d&risl=&pid=ImgRaw&r=0"
img = np.array(Image.open(requests.get(image_url, stream=True).raw))
img = cv2.resize(img, (224, 224))
img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_tensor.shape

breast image

In [ ]:
image_pth = bundles_dir / "breast_density_classification/sample_data/A/sample_A1.jpg"
img = np.array(Image.open(image_pth))
img = cv2.resize(img, (299, 299))
print(np.max(img))
print(np.min(img))
# img = np.float32(img) / np.max(img)
input_tensor = preprocess_image(img)#, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(input_tensor.min())
print(input_tensor.max())
input_tensor.shape

# Define CAM list

In [ ]:
# from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, EigenGradCAM, AblationCAM, RandomCAM
from pytorch_grad_cam import EigenCAM, \
                              EigenGradCAM, \
                              FullGrad, \
                              GradCAM, \
                              GradCAMElementWise, \
                              GradCAMPlusPlus, \
                              HiResCAM, \
                              LayerCAM, \
                              RandomCAM, \
                              ScoreCAM, \
                              XGradCAM, \
                              AblationCAM


cams_list = [#'AblationCAM', # ran out of RAM
            'GradCAM',
            'HiResCAM',
            'GradCAMElementWise',
            'GradCAMPlusPlus',
            'EigenCAM', # done
            #'EigenGradCAM', # Skipped all layers for AlexNet; figure out
            'RandomCAM',
            'LayerCAM',
             'XGradCAM',
             #'FullGrad', # Skipped all layers for DenseNet161; figure out
             #'ScoreCAM', # ran out of RAM
             ]

# Define model list

In [ ]:

model_path = bundles_dir / "breast_density_classification/models/model.pt"
breast_model = TorchVisionFCModel(model_name="inception_v3", num_classes=4, pool=None) # ref for params -> https://github.com/Project-MONAI/model-zoo/blob/dev/models/breast_density_classification/configs/inference.json
breast_model.load_state_dict(torch.load(model_path))
breast_model.eval()

In [ ]:
from torchvision import models # Only needed for example model, not overall anim code
# model = models.densenet161(pretrained=True)

models_dict = {
                # "wide_resnet101_2": models.wide_resnet101_2(pretrained=True), # this needs to be paired with imagenet like images
                # "densenet161": models.densenet161(pretrained=True), # this needs to be paired with imagenet like images
                "breast_density_classification": breast_model # this needs to be paired with breast density images
              }

## visualize model parameters

In [ ]:
model = models_dict["breast_density_classification"]
layers = [layer for layer, what_is_this in model.named_modules()]
print(len(layers))
parameters = [p for p in model.parameters()]# if p.requires_grad]
len(parameters)

In [ ]:
running_sum = 0

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

for layer_name, layer_module in model.named_modules():

  # print("-"*6 + layer_name + "-"*6)
  # print(type(layer_name))
  # print(type(layer_module))
  # layer_parameters = [parameters = [p for p in model.parameters()]# if p.requires_grad]]
  n_layer_params = sum(p.numel() for p in layer_module.parameters() if p.requires_grad)
  running_sum += n_layer_params

print(running_sum)
print(total_params)

# get target_layer

In [ ]:

def get_target_layers(model):
  """
  target_layers doesn't matter for cam_anim (since we loop through all the layers),
  but it is needed for initializing the GradCAM object.
  This method returns the last layer (that is viable as a target i.e. iterable) of the model.

  """


  target_layers = []
  for _, layer_module in model.named_modules():
    # print(_)
    if _ == "features.Mixed_7c.branch_pool.conv":
      target_layers.append(layer_module)
    # https://stackoverflow.com/questions/1952464/in-python-how-do-i-determine-if-an-object-is-iterable
    try:
      some_object_iterator = iter(layer_module)
      if type(layer_module)!=str:
        target_layers.append(layer_module)

    except TypeError as te:
      pass
      # print(some_object, 'is not iterable')
  # target_layers = target_layers[-2]
  print(target_layers)
  return target_layers

# Double Loop - n_models X n_cams

In [ ]:
n_models = len(models_dict.keys())
n_cams = len(cams_list)

print(f"{n_models} X {n_cams} = {n_models*n_cams} iterations")

In [ ]:
import ffmpeg
def _ffmpeg_standard_quality(tmp_path, output_fname, frame_rate=5):
  """ _ffmpeg_standard_quality
      Generates and saves-to-file the animated .MP4 video in standard quality.
      ---
      Input: tmp_path <str>, the path to the images
             output_fname <str>, the path and filename for the saved file
             frame_rate=5 <int>, the number of frames per second
      Output: None
  """
  print('Generating video with pngs from:\n', tmp_path)
  try:
    (
      ffmpeg
      .input(tmp_path + '*.png', pattern_type='glob', framerate=frame_rate)
      .output(output_fname)
      .run()
    )
  except Exception as exc:
    print('ERROR: ffmpeg video generation failed; video corrupt.')
    print(exc)
  return None

In [ ]:
!ffmpeg -version

In [ ]:
metrics_records = []
for model_name, model in models_dict.items():

  print("="*33 + model_name + "="*33)

  outdir = Path(drive_path) / "starCAManim" / model_name
  outdir.mkdir(exist_ok=True, parents= True)

  target_layers = get_target_layers(model)
  # target_layers = ["features.Mixed_7c.branch_pool.conv"]

  for cam_name in cams_list:
    print("-"*33 + cam_name + "-"*33)

    frames_dir = outdir / cam_name
    frames_dir.mkdir(exist_ok=True, parents= True)
    metrics = {"model_name": model_name, "CAM Name": cam_name, "error": None}
    for norm_type in ["global", "layer"]:
      tmp_path = str(outdir / cam_name / norm_type)
      output_fname = str(outdir / f"{cam_name}_{norm_type}_anim.mp4")
      _ffmpeg_standard_quality(tmp_path, output_fname, frame_rate=24)

In [ ]:
from pytorch_grad_cam.base_cam import BaseCAM
import gc
gc.enable()

In [ ]:
print(gc.collect())
print(pd.DataFrame.from_records(gc.get_stats()))

In [ ]:
# %%mprun -f BaseCAM.cam_anim

metrics_records = []
for model_name, model in models_dict.items():

  print("="*33 + model_name + "="*33)

  outdir = Path(drive_path) / "starCAManim" / model_name
  outdir.mkdir(exist_ok=True, parents= True)

  target_layers = get_target_layers(model)
  # target_layers = ["features.Mixed_7c.branch_pool.conv"]

  for cam_name in cams_list:
    print("-"*33 + cam_name + "-"*33)

    frames_dir = outdir / cam_name
    frames_dir.mkdir(exist_ok=True, parents= True)
    metrics = {"model_name": model_name, "CAM Name": cam_name, "error": None}
    try:
      cam = globals()[cam_name](model=model, target_layers=target_layers, use_cuda=False)
      metrics_update = cam.cam_anim(img,
                                    input_tensor,
                                    frame_rate=24,
                                    norm_type='both',
                                    keep_frames=True,
                                    tmp_dir=str(frames_dir),
                                    output_fname=str(outdir / f"{cam_name}_anim.mp4"))

      metrics.update(metrics_update)
      del cam
      gc.collect()
      print(pd.DataFrame.from_records(gc.get_stats()))
      print(f"{cam_name} Finished")


    except Exception as ex:
      metrics.update({"error": str(ex)})
      print(ex)
      print("Continuing to the next CAM")
      raise ex

    metrics_records.append(metrics)

    with open(str(Path(drive_path) / "starCAManim" / f"metrics_logs_{model_name}.json"), "w") as outfile:
      json.dump(metrics_records, outfile, indent=4)

# Load logged metrics

In [ ]:

# Opening JSON file
model_name = "breast_density_classification"
with open(str(Path(drive_path) / "starCAManim" / f"metrics_logs_{model_name}.json"), 'r') as openfile:
    # Reading from json file
    metrics_records = json.load(openfile)

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame.from_records(metrics_records)
display(metrics_df)

In [ ]:
def unpack_layer_records(row):

  layer_df = pd.DataFrame.from_records(row["layers_records"])

  for col in row.index:
    if col not in ["layers_records", "error"]:
      layer_df[col] = row[col]

    layer_df["cam_anim_error"] = row["error"]

  return layer_df


In [ ]:
import pandas as pd
df = pd.DataFrame({
  'date': ['2022-09-14', '2022-09-15', '2022-09-16'],
  'letter': ['A', 'B', 'C'],
  'dict' : [{ 'fruit': 'apple', 'weather': 'aces'},
            { 'fruit': 'banana', 'weather': 'bad'},
            { 'fruit': 'cantaloupe', 'weather': 'cloudy'}],
})

pd.concat([df.drop(['dict'], axis=1), df['dict'].apply(pd.Series)], axis=1)

In [ ]:
layer_metrics_df = None

for _, row in metrics_df.iterrows():
  layer_df = unpack_layer_records(row)

  if layer_metrics_df is None:
    layer_metrics_df = layer_df
  else:
    layer_metrics_df = pd.concat([layer_metrics_df, layer_df])


display(layer_metrics_df)
# layer_metrics_df = layer_metrics_df.groupby("CAM Name")
layer_metrics_df["layer_time"].describe()

# Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd

df = layer_metrics_df.copy(deep=True)
# df = df[df["CAM Name"]=="HiResCAM"]

plt.figure(figsize=(9,9))
sns.scatterplot(x="layer_num_parameters", y="layer_time",
                # s=100,
                # figsize=(64,64),
                hue='CAM Name',
                #style='zone_number',
                data=df)

plt.xscale('log')
plt.yscale('log')
plt.show()